### Solid Waste Engineering 2018
#### Book 2 of x see notebook Solid Waste Engineering 2018
#### Semester project: Testing the probability of garbage
Masters students in Environmental Engineering from the [École Polytechnique Féderale de Lausanne](https://enac.epfl.ch/environmental-engineering) test the hypothesis that litter densities on Lac Léman are predicatable.  The current method is based on the Probability Density Function derived from the logarithm of the pieces/meter of trash (pcs/m) from over 100 samples.

#### Purpose: Understanding how the results from one inventory relate to the rest of the inventories for the lake.

1. Getting data from the API
2. Reporting descriptive statistics
3. Comparing results on the lake
4. Graphing results

We will add to this workbook as the semester advances

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import re
from scipy.stats import norm
import scipy.stats
import statsmodels.api as sm
import seaborn
import requests
import json

### This is all the daily totals in the database for Lac Léman:
The api makes this operation very straight forward it can be put into a dataFrame directly.

In [2]:
# make a function to get data
# we will hit the api more than once
def getData(url):
    a = url
    c = requests.get(url).json()
    d = pd.DataFrame(c)
    return d

In [3]:
this_data = getData("http://mwshovel.pythonanywhere.com/dirt/daily-total/Lac-Léman/?format=json")
# the pcs/m ratio is not stored in the database and therefore needs to be calculated
this_data["pcs_m"]=this_data.total / this_data.length
# make a copy and give it a name that is easier to recognize:
dailyTotalsLac = this_data.copy()
colNameTotals = list(dailyTotalsLac.columns)
# one record from the df
dailyTotalsLac.iloc[0]

date                2015-11-23
length                      61
location    Baye-de-Montreux-G
total                      349
pcs_m                  5.72131
Name: 0, dtype: object

### This is all the beaches in the database for Lac Léman:
This gives all the geo information for each beach in the data base:

In [4]:
beach_data = getData("http://mwshovel.pythonanywhere.com/dirt/beaches/Lac-L%C3%A9man/")
beachData = beach_data.copy()
beachColumns = list(beachData.columns)
# one record from the df
beachData.iloc[0]

city          La-Tour-de-Peilz
latitude           46.44721600
location         Anarchy-Beach
longitude           6.85961200
owner                 mwshovel
post                      1814
project_id                MCBP
water                        l
water_name           Lac-Léman
Name: 0, dtype: object

### Getting the object information for each inventory
The inventories are stored in a non aggregated form.

In [5]:
code_data = getData("https://mwshovel.pythonanywhere.com/dirt/codes/Lac-Léman/")
codeData = code_data.copy()
codeColumns = list(codeData.columns)
codeData.iloc[0]

code_id                      G100
date                   2015-11-23
length                         61
location_id    Baye-de-Montreux-G
owner                    mwshovel
project_id                   MCBP
quantity                        1
Name: 0, dtype: object

### This gets the MLW codes from the website:

In [6]:
csvUrl = "https://mwshovel.pythonanywhere.com/static/newCriteria.csv"
mlwCodes = pd.read_csv(csvUrl)

### The descriptive statistics for each beach on the lake and for the lake
This data is available on the api also

In [7]:
# make a function to get all the results in a dictionary:
url_x = "http://mwshovel.pythonanywhere.com/dirt/summary" #<--- need to add the following "/beach or lake/?format=json"
beachNames = list(beachData.location)
cities = list(beachData.city.unique())


In [8]:
def getSummaries(aList):
    c = {}
    for x in aList:
        a = str(url_x + "/" + str(x) + "/?format=json")
        b = requests.get(a).json()
        d = {x:b}
        c.update(d)
    return c
beachSummaries = getSummaries(beachNames)
citySummaries = getSummaries(cities)
lakeSummary = getSummaries(['Lac-Léman'])

In [9]:
# so if that works we should be able to do soemthing like this:
print(cities[3])
print(citySummaries[cities[3]])
# that looks good!

Tolochenaz
{'num_locs': 1, 'total': 402, 'num_samps': 1, 'first': '2016-04', 'last': '2016-04', 'ave_dense': '3.2683', 'max_dense': '3.2683', 'min_dense': '3.2683', 'two_five': '3.2683', 'seven_five': '3.2683', 'num_lakes': 1, 'num_rivers': 0, 'stan_dev': '0.0000', 'location': 'Tolochenaz'}
